In [10]:
import numpy as np
import pandas as pd
import os

In [8]:
res_path = 'results/ENZYMES/1.csv'
res_df = pd.read_csv(res_path)
means = res_df.iloc[:, 1:].mean()
stds = res_df.iloc[:, 1:].std()
print(means)
print(stds)

train_loss               0.441026
train_accuracy           0.845000
validation_loss          1.182600
validation_accuracy      0.815000
training_time          551.128436
dtype: float64
train_loss             0.246570
train_accuracy         0.095888
validation_loss        1.680288
validation_accuracy    0.168956
training_time          4.016355
dtype: float64


In [37]:
import pandas as pd
import os

# Initialize a list to store the results
results = []

# Loop over all csv files in the directory and its subdirectories
for root, dirs, files in os.walk('results/'):
    for file in files:
        if file.endswith('.csv'):
            # Read the csv file
            res_path = os.path.join(root, file)
            res_df = pd.read_csv(res_path)

            # Calculate the mean and standard deviation
            means = res_df.iloc[:, 1:].mean()
            stds = res_df.iloc[:, 1:].std()

            # Prepare the result dictionary
            result = {'config_id': int(file.rstrip('.csv'))}  # remove '.csv' from the file name
            
            # Append each mean and std as a new column
            for column in res_df.columns[1:]:
                result[column + '_mean'] = means[column]
                result[column + '_std'] = stds[column]
                        
            # Append the result to the list
            results.append(result)

            # # Append the results to the list
            # results.append({
            #     'config_id': file.rstrip('.csv'),  # remove '.csv' from the file name
            #     'train_accuracy': f"{means['train_accuracy']:.3f}({stds['train_accuracy']:.3f})",
            #     'validation_accuracy': f"{means['validation_accuracy']:.3f}({stds['validation_accuracy']:.3f})"
            # })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results).set_index('config_id')

In [38]:
import yaml
import pandas as pd
import os

# Initialize a list to store the results
results = []

# Loop over all yaml files in the directory
for file in os.listdir('results/'):
    if file.endswith('.yml'):
        # Read the yaml file
        res_path = os.path.join('results/', file)
        with open(res_path, 'r') as stream:
            try:
                # Parse the yaml file
                yaml_content = yaml.safe_load(stream)
                                
                # Append the yaml content to the list
                results.append(yaml_content)
            except yaml.YAMLError as exc:
                print(exc)

# Convert the results to a DataFrame
config_df = pd.DataFrame(results).set_index('config_id')



In [41]:
res_config_df = results_df.join(config_df, how='inner')

In [57]:
res_config_df.to_csv('results/aggr_all.csv')

In [54]:
output_df = res_config_df[['data', 'pde_type']].copy()
# List of columns to format
columns_to_format = [ 'train_accuracy', 'validation_accuracy', 'training_time']

# Iterate through the columns and format them
for column in columns_to_format:
    output_df[column] = res_config_df[column + '_mean'].map("{:.3f}".format) + "(" + res_config_df[column + '_std'].map("{:.3f}".format) + ")"

In [58]:
output_df.to_csv("results/aggr_output.csv")

In [61]:
# Convert the DataFrame to a LaTeX table
latex_table = output_df.to_latex(index=False)

In [63]:
print(latex_table)

\begin{tabular}{lllll}
\toprule
    data & pde\_type & train\_accuracy & validation\_accuracy &    training\_time \\
 ENZYMES &     heat &   0.845(0.096) &        0.815(0.169) &   551.128(4.016) \\
\midrule
 ENZYMES &     wave &   0.799(0.140) &        0.785(0.214) &   399.866(2.948) \\
PROTEINS &     heat &   0.791(0.022) &        0.536(0.040) & 1045.004(13.007) \\
PROTEINS &     wave &   0.779(0.024) &        0.705(0.072) &  857.061(11.078) \\
   MUTAG &     wave &   0.869(0.040) &        0.830(0.073) &   590.860(2.783) \\
   MUTAG &     heat &   0.864(0.032) &        0.787(0.094) &   218.688(0.732) \\
\bottomrule
\end{tabular}

